In [72]:
from pathlib import Path
import pandas as pd
import altair as alt
import seaborn as sns

In [73]:
Path.cwd()

PosixPath('/Users/ben/Developer/SpaceMissionDES/SpaceMissionDES/analysis')

#### Load the Data

In [89]:
data_dir = Path.cwd().parent.parent / "results" / "Case04-r02"

mc_results = pd.read_csv(data_dir / "mc.csv")
mc_results.head()

,replicant,outcome,duration,anomaly_count,anomaly_time,anomaly_vehicle,anomaly_activity
0,0,True,106.000000,0,NaN,NaN,NaN
1,1,True,115.963867,2,0.000000,MTV,Countdown
2,1,True,115.963867,3,3.713458,MTV,Countdown
3,2,True,106.000000,0,NaN,NaN,NaN
4,3,True,132.197189,2,0.000000,MTV,Countdown


#### Summary Statistics

In [90]:
N = mc_results.replicant.max()
success_rate = sum(mc_results.groupby("replicant").first().outcome) / N
print(f"The success rate was: {success_rate} = {100*success_rate:.3f} %\n")

The success rate was: 1.001001001001001 = 100.100 %



#### Mission Duration

In [91]:
df = mc_results.groupby("replicant", as_index=False).first()[["replicant", "duration"]]

alt.Chart(df).mark_bar().encode(
    alt.X("duration:Q", bin=True),
    # alt.X("duration:Q", bin=alt.Bin(extent=[0, 400], step=25)),
    y='count()',
).properties(
    width = 800
)

alt.Chart(...)

Compute the Emprical CDF

In [92]:
import numpy as np

# from: https://stackoverflow.com/questions/33345780/empirical-cdf-in-python-similiar-to-matlabs-one
def ecdf(sample):

    # convert sample to a numpy array, if it isn't already
    sample = np.atleast_1d(sample)

    # find the unique values and their corresponding counts
    quantiles, counts = np.unique(sample, return_counts=True)

    # take the cumulative sum of the counts and divide by the sample size to
    # get the cumulative probabilities between 0 and 1
    cumprob = np.cumsum(counts).astype(np.double) / sample.size

    return quantiles, cumprob

def plot_ECDF(df):
    sample = df.duration.values
    qe, pe = ecdf(sample)
    data = pd.DataFrame({"duration": list(qe), "prob": list(pe)})
    fig = alt.Chart(data).mark_line().encode(x = "duration", y = "prob").properties(width=500, height=250)
    return fig


In [93]:
fig = plot_ECDF(df)
fig.interactive()

alt.Chart(...)

#### Failure Diagnostics (Aggregate)

Which Vehicles Fail

In [79]:
# Relative failure frequency by vehicle
df = (
    mc_results
    .groupby("anomaly_vehicle")
    .count()
    .rename(columns={'replicant': 'n'})
    .loc[:,'n']
)

fail_count = df.sum()
# (df / fail_count).reset_index()

alt.Chart(
    (df / fail_count).reset_index()
).mark_bar().encode(
    y = "anomaly_vehicle",
    x = "n"
)

alt.Chart(...)

Which Activities Fail?

In [80]:
# Relative failure frequency by activity
df = (
    mc_results
    .groupby("anomaly_activity")
    .count()
    .rename(columns={'replicant': 'n'})
    .loc[:,'n']
)

fail_count = df.sum()
# (df / fail_count).reset_index()

alt.Chart(
    (df / fail_count).reset_index()
).mark_bar().encode(
    y = "anomaly_activity",
    x = "n"
)

alt.Chart(...)

#### Failure Diagnostics (per mission)

Avg ascent failures per mission

In [81]:
mc_results.head()

,replicant,outcome,duration,anomaly_count,anomaly_time,anomaly_vehicle,anomaly_activity
0,0,True,298.150349,2,0.000000,MTV,Countdown
1,1,False,288.041788,2,288.041788,MTV,Mars Checkout
2,2,True,288.021517,0,NaN,NaN,NaN
3,3,True,288.160772,0,NaN,NaN,NaN
4,4,True,288.291936,0,NaN,NaN,NaN


In [82]:
mc_results.query("anomaly_count > 1")
# df = mc_results.query("replicant == 3")
# df

,replicant,outcome,duration,anomaly_count,anomaly_time,anomaly_vehicle,anomaly_activity
0,0,True,298.150349,2,0.000000,MTV,Countdown
1,1,False,288.041788,2,288.041788,MTV,Mars Checkout
5,5,True,295.104570,2,0.000000,MTV,Countdown
9,9,True,309.273624,2,0.000000,MTV,Countdown
10,9,True,309.273624,3,7.369420,MTV,Ascent
...,...,...,...,...,...,...,...
1073,989,True,326.215029,2,0.000000,MTV,Countdown
1074,989,True,326.215029,3,9.121076,MTV,Countdown
1075,989,True,326.215029,4,21.542724,MTV,Orbit Insertion
1082,996,True,296.586868,2,0.000000,MTV,Countdown


In [83]:
df.anomaly_activity.value_counts()

AttributeError: 'Series' object has no attribute 'anomaly_activity'

In [ ]:
df = mc_results.groupby("replicant", as_index=False)["anomaly_activity"].value_counts()
df

,replicant,anomaly_activity,count
0,31,Orbit Insertion,1
1,79,Mars Transit,1
2,91,Ascent,1
3,144,Mars Checkout,1
4,183,Orbit Insertion,1
5,305,Mars Checkout,1
6,350,Ascent,1
7,409,Ascent,1
8,411,Ascent,1
9,464,Mars Transit,1


In [ ]:
# alt.Chart(df.unstack().Ascent.reset_index()).mark_boxplot().encode(x = "Ascent")

In [ ]:
df = mc_results.groupby("replicant", as_index=False)["anomaly_activity"].value_counts()

alt.Chart(df).mark_boxplot().encode(
    x = "count",
    y = "anomaly_activity"

)

alt.Chart(...)

In [ ]:
mc_results

,replicant,outcome,duration,anomaly_count,anomaly_time,anomaly_vehicle,anomaly_activity
0,0,True,288.0,0,NaN,NaN,NaN
1,1,True,288.0,0,NaN,NaN,NaN
2,2,True,288.0,0,NaN,NaN,NaN
3,3,True,288.0,0,NaN,NaN,NaN
4,4,True,288.0,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...
995,995,True,288.0,0,NaN,NaN,NaN
996,996,True,288.0,0,NaN,NaN,NaN
997,997,True,288.0,0,NaN,NaN,NaN
998,998,True,288.0,0,NaN,NaN,NaN
